In [1]:
from ansatz_simulation_class import AnsatzSimulation
import torch
import math
from genetic_quantum import QuantumModel, QuanvLayer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
from patch_making import PatchExtraction, Quanv_2d
from torch import tensor
import matplotlib.pyplot as plt
from sampled_cv_dataset import SampledDataset4Training
import time
from tqdm import tqdm

### Defining Hyrbid Module class

In [15]:
class L2NormalizationLayer(nn.Module):
    def __init__(self, dim=1, eps=1e-12):
        super(L2NormalizationLayer, self).__init__()
        self.dim = dim
        self.eps = eps

    def forward(self, x):
        return F.normalize(x, p=2, dim=self.dim, eps=self.eps)

class HybridModel(nn.Module):
    def __init__(self, n_qubits, patch_size, chromosome, num_classes, input_size, ansatz_parameters, mode='2d'):
        super().__init__()
        self.quanv_layer = QuanvLayer(
            n_qubits=n_qubits,
            patch_size=patch_size,
            chromosome=chromosome,
            parameters = ansatz_parameters,
            mode=mode
        )
        feature_size = input_size
        self.fc1 = nn.Linear(feature_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.norm = nn.LayerNorm(feature_size)
        #self.conv1 = nn.Conv2d(in_channels = 1, output_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=patch_size, stride=patch_size)
        self.fc = nn.Linear(feature_size, num_classes)
        self.l2norm = L2NormalizationLayer(dim=1)
        
    def forward(self, x):
        print("Passing through quanvolution layer...")
        start_time = time.perf_counter()
        x = self.quanv_layer.forward(x)
        end_time = time.perf_counter()
        print(f'Quanvolution processing time: {end_time - start_time}')  
        x = x.flatten(start_dim=1)
        #print(x.shape)
        x = self.l2norm(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

### Instantiating genetic ansatz model

In [30]:
n_qubits = 4
max_patches = 80
patch_size = 2
batch_size = 100
num_classes = 2
input_size = max_patches * n_qubits
num_epochs = 4
toy_chromosome = [['ctrl_0', 'trgt_0', 'ctrl_1', 'trgt_1'], [None, 'ctrl_0', 'trgt_0', None], ['ry_gate', 'rz_gate', 'ry_gate', 'rx_gate'],['trgt_0', 'ctrl_0', 'trgt_1', 'ctrl_1'], ['rz_gate', 'trgt_0', 'ctrl_0', 'ry_gate']]
#simple_fc = ClassicalComponent(num_classes, input_size)
params = torch.rand(6)*math.pi
hqcnn = HybridModel(n_qubits, patch_size, toy_chromosome, num_classes, input_size, params)
hqcnn

HybridModel(
  (quanv_layer): QuanvLayer()
  (fc1): Linear(in_features=320, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=2, bias=True)
  (l2norm): L2NormalizationLayer()
)

In [ ]:
print(f'Parameters in the random circuit as angles: {torch.rad2deg(params)}')

Parameters in the random circuit: tensor([123.3380, 161.1354, 121.5233, 145.8758, 169.8532,  72.2481])


### Getting data and sampling it

In [11]:
def sample_classes(target_classes, dataset):
    return [dataset.__getitem__(index) for index in range(dataset.__len__()) if dataset.__getitem__(index)[1] in target_classes]

In [12]:
def sample_data(dataset, classes, sample_size):
    class_one = 0
    class_two = 0
    binary_dataset = []

    for image in dataset:
        if image[1] == classes[0] and class_one < sample_size:
            binary_dataset.append(image)
            class_one +=1 
        elif image[1] == classes[1] and class_two < sample_size:
            binary_dataset.append(image)
            class_two += 1
            
        if class_one == sample_size and class_two == sample_size:
            break
    
    return binary_dataset

In [13]:
def create_dataset(dataset_class, transform_routine, target_classes, sample_size):
    training_dataset = dataset_class(root='./data', train=True, download=True, transform=transform_routine)
    validation_dataset = dataset_class(root='./data', train=False, transform=transform_routine, download=True)
    
    reduced_training_dataset = sample_classes(target_classes, training_dataset)
    reduced_validation_dataset = sample_classes(target_classes, validation_dataset)
    
    sampled_training_dataset = sample_data(reduced_training_dataset, target_classes, sample_size)
    sampled_validation_dataset = sample_data(reduced_validation_dataset, target_classes, sample_size)
    
    return SampledDataset4Training(sampled_training_dataset), SampledDataset4Training(sampled_validation_dataset) 

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    PatchExtraction(patch_size, max_patches)])

target_classes = [0, 1]

sample_size = 200

training_data, validation_data = create_dataset(datasets.MNIST, transform, target_classes, sample_size)

### Training my model

In [17]:
def train_model(model, train_loader, num_epochs, optimizer, loss_fn, filepath):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        epoch_loss = 0.0
        total = 0
        
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, (inputs, labels) in progress_bar:
        
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)  # Calculate loss
            
            running_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            loss.backward()  # Backpropagate to calculate gradients
            optimizer.step() # Update weights
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/(i+1):.4f}, Acc: {correct / total:.4f}")
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: Train Loss={epoch_loss:.4f}, Train Acc={epoch_acc:.4f}")
            # Print every 10 batches
            
    torch.save(model.state_dict(), filepath)    

In [32]:
#quantum_processing = transforms.Compose([Quanv_2d(n_qubits, toy_chromosome)])
#training_zeros_ones = SampledDataset4Training(binary_dataset)
training_loader = DataLoader(training_data, batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size, shuffle=True)

In [20]:
post_processing = [label for (data, label) in validation_loader]

In [21]:
post_processing

[tensor([1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
         1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
         1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
         0, 1, 1, 0]),
 tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
         1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
         0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
         0, 1, 0, 1]),
 tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
         1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 

In [33]:
optimizer = torch.optim.Adam(hqcnn.parameters(), lr=0.01, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()
#loss = loss_fn(output, mnist_labels)
device = torch.device("cpu")

In [34]:
filepath = "C:/Users/speak/Genetic-Algorithm-based-Optimization-for-Quantum-Circuit-Synthesis/src/models/hqcnn_mnist_weights_with_random_params.pth"

train_model(hqcnn, training_loader, num_epochs, optimizer, loss_fn, filepath)

Epoch 1/4:   0%|          | 0/4 [00:00<?, ?it/s]

Passing through quanvolution layer...


Epoch 1/4:  25%|██▌       | 1/4 [00:03<00:11,  3.90s/it]

Quanvolution processing time: 3.9074244999792427
Epoch [1/4], Step [1/4], Loss: 0.6938, Acc: 0.5200
Passing through quanvolution layer...


Epoch 1/4:  50%|█████     | 2/4 [00:08<00:08,  4.50s/it]

...

Quanvolution processing time: 4.6229619999649
Epoch [4/4], Step [2/4], Loss: 0.6152, Acc: 0.7650
Passing through quanvolution layer...


Epoch 4/4:  75%|███████▌  | 3/4 [00:14<00:04,  4.71s/it]

Quanvolution processing time: 4.77374360000249
Epoch [4/4], Step [3/4], Loss: 0.6067, Acc: 0.7900
Passing through quanvolution layer...


Epoch 4/4: 100%|██████████| 4/4 [00:18<00:00,  4.67s/it]

Quanvolution processing time: 4.5730526999686845
Epoch [4/4], Step [4/4], Loss: 0.6038, Acc: 0.8050
Epoch 4: Train Loss=0.6038, Train Acc=0.8050


#### Evaluating feature space

In [44]:
mean = 0.
meansq = 0.
for (data, label) in training_loader:
    mean = data.mean()
    meansq = (data**2).mean()

std = torch.sqrt(meansq - mean**2)
print("mean: " + str(mean))
print("std: " + str(std))

mean: tensor(0.4046)
std: tensor(0.4424)


In [45]:
post_processing = [data for (data, label) in training_loader]

In [46]:
post_process = torch.stack(post_processing).squeeze(0)
post_process.shape

torch.Size([4, 100, 50, 4])

In [47]:
mean_p = torch.mean(post_process)
mean_p

tensor(0.4163)

In [48]:
p_diffs = post_process - mean_p
p_var = torch.mean(torch.pow(p_diffs, 2.0))
p_var

tensor(0.1968)

In [49]:
p_std = torch.pow(p_var, 0.5)
p_std

tensor(0.4436)

In [50]:
phi_n = F.normalize(post_process.flatten(start_dim=1), dim=1)
cos_sim = phi_n @ phi_n.T
cos_sim.mean()

tensor(0.6441)

In [53]:
torch.cuda.is_available()

False

### Testing model

In [34]:
def test_model(model, test_loader, hyperparams, num_epochs, optimizer, loss_fn, output_file):
    model.eval()
    
    total_loss = 0.0
    all_labels = []
    all_predictions = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item(outputs, dim=1)[:, 1]
            
            probs = torch.softmax()
            all_probs.extend(probs)
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    avg_loss = total_loss / len(test_loader)
    
    with open(output_file, "a") as f:
        f.write(f"\n[Quantum Model Testing - {time.perf_counter()}]\n")
        f.write(f"Hyperparameters: {hyperparams}\n")
        f.write(f"Test Loss: {avg_loss:.4f}\n")
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"AUC: {auc:.4f}\n")
        f.write("-" * 50 + "\n")

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    return avg_loss, accuracy, precision, recall, f1, auc


### Validating model

In [35]:
def validate_model(model, dataloader, criterion, device):
    model.eval()    
    model.to(device)

    total_loss = 0.0
    all_labels = []
    all_predictions = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
#            images = images.to(device)
#            labels = labels.squeeze().to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy() if outputs.shape[1] > 1 else torch.softmax(outputs, dim=1)[:, 0].cpu().numpy()
            all_probs.extend(probs)
            all_predictions.extend(outputs.argmax(dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    f1 = f1_score(all_labels, all_predictions, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auc = float("nan")

    avg_loss = total_loss / len(dataloader)
    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    return avg_loss, accuracy, precision, recall, f1, auc



In [36]:
validate_model(hqcnn, validation_loader, loss_fn, device)

Validation:   0%|          | 0/4 [00:00<?, ?it/s]

Passing through quanvolution layer...


Validation:  25%|██▌       | 1/4 [00:04<00:14,  4.73s/it]

Quanvolution processing time: 4.72599090001313
Passing through quanvolution layer...


Validation:  50%|█████     | 2/4 [00:09<00:09,  4.98s/it]

...

Validation:  75%|███████▌  | 3/4 [00:15<00:05,  5.08s/it]

Quanvolution processing time: 5.20073370001046
Passing through quanvolution layer...


Validation: 100%|██████████| 4/4 [00:20<00:00,  5.06s/it]

Quanvolution processing time: 5.119025400024839
Validation Loss: 0.5769
Accuracy: 0.9450, Precision: 0.9045, Recall: 0.9950, F1: 0.9476, AUC: 0.9983


(0.5768589973449707,
 0.945,
 0.9045454545454545,
 0.995,
 0.9476190476190476,
 0.99835)

In [54]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy().reshape(28, 28)
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

ValueError: cannot reshape array of size 200 into shape (28,28)